In [76]:
import numpy as np
import matplotlib.pyplot as plt

In [77]:
from loaders import clean_text_for_pattern_letter
from temp_text_puh import template_str
# input_str = clean_text_for_pattern_letter(template_str, out_len=len(template_str))
input_str = clean_text_for_pattern_letter(template_str, out_len=5000)
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
               'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', '\'', '1']
print(f'{letters=}')

len(letters)=32
5000
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', "'", '1']


In [78]:
def predict_train(input_str,
                  letters,
                  max_history=4,
                  max_index: int = 4,):
    WIN: dict[str, dict[str, int]] = {}

    HistoryHW: list[dict[str, int]] = []
    HistoryW: list[dict[str, dict[str, dict[int, int]]]] = []

    for h_idx in range(max_history):
        HistoryHW.append({})
        HistoryW.append({})

    IDX: list = []

    for letter in letters:
        WIN.setdefault(letter, {})

        for idx in range(max_index):
            WIN[letter].setdefault(idx, 0)

    prev_indices = [0]

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0:
            WIN[inp][prev_indices[0]] += 1
            continue

        IDX.append(prev_indices[0])

        IdxO = {}

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            HW = HistoryHW[h_idx]
            W = HistoryW[h_idx]

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            if HW.get(key_HW, -1) != -1:
                inp_has_index = HW[key_HW]

                # idx_strength = W[prev_inp + str(prev_indices[h_idx])][inp][inp_has_index]

                IdxO.setdefault(inp_has_index, 0)
                IdxO[inp_has_index] += 1

        new_index = 0
        if len(IdxO) == 0:
            new_index = min(WIN[inp], key=WIN[inp].get)
        else:
            new_index = max(IdxO, key=IdxO.get)
        WIN[inp][new_index] += 1

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            key_I = prev_inp + str(prev_indices[h_idx])
            HistoryW[h_idx].setdefault(key_I, {})
            HistoryW[h_idx][key_I].setdefault(inp, {})
            HistoryW[h_idx][key_I][inp].setdefault(new_index, 0)

            stats = HistoryW[h_idx][key_I][inp][new_index]
            if stats == 0:
                HistoryW[h_idx][key_I][inp][new_index] += 1
            else:
                HistoryW[h_idx][key_I][inp][new_index] += 1 / (32 * stats)

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            HistoryHW[h_idx].setdefault(key_HW, new_index)

        # recurtion
        prev_indices.insert(0, new_index)
        if len(prev_indices) > max_history:
            prev_indices = prev_indices[:-1]
    
    IDX.append(new_index)

    return HistoryW, HistoryHW, IDX

max_index_L1 = 16
max_history_L1 = 4

HistoryW, HistoryHW, IDX = predict_train(input_str, letters, max_history=max_history_L1, max_index=max_index_L1)

print(f'{len(IDX)}')

for h_idx in range(max_history_L1):
    print(f'{h_idx=}')
    for wk, wv in HistoryW[h_idx].items():
        print(wk, wv)


5000
h_idx=0
c0 {'h': {0: 1.1196347139148715}, 'i': {11: 1}}
h0 {'a': {0: 1.1196347139148715, 14: 1}, 'e': {14: 1.1196347139148715}, 't': {10: 1.03125}, 'i': {5: 1.0909910320871516}, 'r': {8: 1}, ' ': {5: 1.0909910320871516}}
a0 {'p': {0: 1.03125}, ' ': {15: 1.0615530303030303}, 'n': {7: 1.03125, 0: 1.03125}, 'u': {6: 1}, 'm': {15: 1.03125}, 'i': {4: 1}, 'd': {11: 1.1196347139148715, 13: 1}, 's': {14: 1}, 'c': {11: 1}, 'b': {3: 1.03125}}
p0 {'t': {0: 1}, 'o': {2: 1}, ' ': {3: 1}, 'p': {6: 1}}
t0 {'e': {0: 1}, 'h': {4: 1.2528507280790113}, 'o': {15: 1, 0: 1.03125}, ' ': {9: 1.0909910320871516}}
e0 {'r': {0: 1.2273902022056227}, 'n': {5: 1.1475456027178208}, ' ': {11: 1.2273902022056227, 9: 1.03125, 10: 1}, 'l': {5: 1.03125}, 'i': {1: 1}, 'a': {11: 1}, 'e': {0: 1.03125}, 't': {7: 1}, 'y': {9: 1}}
r0 {' ': {0: 1.3729611265024908}}
 0 {'i': {0: 1.1196347139148715}, 'h': {6: 1.1196347139148715, 0: 1}, 'b': {7: 1.1747776359598883}, 'r': {7: 1.201378414731495}, 'o': {9: 1.03125}, 'g': {10: 1.

In [79]:
def reverse_predict_train(input_str: list,
                          IDX: list,
                          max_history=4,):
    HistoryW: list[dict[str, dict[str, dict[int, int]]]] = []
    
    for h_idx in range(max_history):
        HistoryW.append({})

    prev_indices = [0]

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0:
            continue

        new_index = IDX[inp_idx]

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            key_I = prev_inp + str(prev_indices[h_idx])
            HistoryW[h_idx].setdefault(key_I, {})
            HistoryW[h_idx][key_I].setdefault(inp, {})
            HistoryW[h_idx][key_I][inp].setdefault(new_index, 0)

            stats = HistoryW[h_idx][key_I][inp][new_index]
            if stats == 0:
                HistoryW[h_idx][key_I][inp][new_index] += 1
            else:
                HistoryW[h_idx][key_I][inp][new_index] += 1 / (32 * stats)

        # recurtion
        prev_indices.insert(0, new_index)
        if len(prev_indices) > max_history:
            prev_indices = prev_indices[:-1]

    return HistoryW


RevHistoryW = reverse_predict_train(input_str[::-1],
                                    IDX[::-1],
                                    max_history=max_history_L1, )

print(f'{len(IDX)=}')

for h_idx in range(max_history_L1):
    print(f'{h_idx=}')
    for wk, wv in RevHistoryW[h_idx].items():
        print(wk, wv)


len(IDX)=5000
h_idx=0
u0 {'o': {14: 1}, 'm': {7: 1}, 's': {6: 1}, ' ': {10: 1}, 'a': {1: 1}, 'd': {0: 1}}
o14 {'y': {2: 1.0615530303030303}, 't': {7: 1}, 'f': {12: 1.03125}, 'p': {10: 1.0909910320871516}, ' ': {8: 1.0615530303030303}, 'r': {6: 1}, 'c': {6: 1.03125}, 'b': {12: 1.03125}}
y2 {' ': {11: 1.03125, 0: 1}, 't': {5: 1}, 'l': {3: 1.0615530303030303}, 'e': {2: 1.03125, 1: 1.03125}}
 11 {'o': {4: 1.1475456027178208}, 'e': {6: 1.2273902022056227, 0: 1.2273902022056227}, 'd': {10: 1.1475456027178208}, 'u': {1: 1.03125}, 'y': {13: 1.0909910320871516}, 'f': {2: 1.1196347139148715, 8: 1}, 't': {4: 1}, 'm': {0: 1}, 'h': {8: 1}, 's': {1: 1}, 'p': {12: 1}, 'n': {10: 1.03125}}
o4 {'s': {0: 1.1475456027178208}, 'g': {15: 1.0615530303030303}, 'w': {15: 1}, 'c': {7: 1.03125}, 'f': {6: 1.0615530303030303}, 't': {6: 1.03125}}
s0 {' ': {10: 1.2273902022056227}, 'e': {1: 1.03125}, 't': {11: 1}, 'a': {12: 1.03125}, 'l': {1: 1.03125}}
 10 {'d': {2: 1.1475456027178208}, 'o': {15: 1.2273902022056227,

In [80]:
def to_probability(HistoryW: list[dict[str, dict[str, dict[int, int]]]], max_history: int):
    for h_idx in range(max_history):
        W = HistoryW[h_idx]

        for inp_symbol, s_indeces in W.items():
            symbol_stat_sum = 0

            for out_symbol, index_stat in s_indeces.items():
                for index, idx_stat in index_stat.items():
                    symbol_stat_sum += idx_stat

            for out_symbol, index_stat in s_indeces.items():
                for index, idx_stat in index_stat.items():
                    W[inp_symbol][out_symbol][index] = idx_stat / symbol_stat_sum
    
    return HistoryW
        
HistoryW = to_probability(HistoryW, max_history=max_history_L1)
RevHistoryW = to_probability(RevHistoryW, max_history=max_history_L1)

for h_idx in range(max_history_L1):
    for wk, wv in RevHistoryW[h_idx].items():
        print(wk, wv)

u0 {'o': {14: 0.16666666666666666}, 'm': {7: 0.16666666666666666}, 's': {6: 0.16666666666666666}, ' ': {10: 0.16666666666666666}, 'a': {1: 0.16666666666666666}, 'd': {0: 0.16666666666666666}}
o14 {'y': {2: 0.12777715074181745}, 't': {7: 0.12036812772824194}, 'f': {12: 0.1241296317197495}, 'p': {10: 0.13132054790063277}, ' ': {8: 0.12777715074181745}, 'r': {6: 0.12036812772824194}, 'c': {6: 0.1241296317197495}, 'b': {12: 0.1241296317197495}}
y2 {' ': {11: 0.16753846153846155, 0: 0.16246153846153846}, 't': {5: 0.16246153846153846}, 'l': {3: 0.17246153846153847}, 'e': {2: 0.16753846153846155, 1: 0.16753846153846155}}
 11 {'o': {4: 0.07638592855579825}, 'e': {6: 0.08170075339378012, 0: 0.08170075339378012}, 'd': {10: 0.07638592855579825}, 'u': {1: 0.06864475680670362}, 'y': {13: 0.07262139546755599}, 'f': {2: 0.0745280510536044, 8: 0.06656461266104594}, 't': {4: 0.06656461266104594}, 'm': {0: 0.06656461266104594}, 'h': {8: 0.06656461266104594}, 's': {1: 0.06656461266104594}, 'p': {12: 0.06

In [81]:
def HO_mx_diff(HO: dict[str, dict[int, int]], 
               RHO: dict[str, dict[int, int]],
               eps = 1.0):
    D: dict[str, dict[int, int]] = {}

    diff_sum = 0

    for symbol, sbl_stats in HO.items():
        rho_sbl = RHO.get(symbol, {})
        D.setdefault(symbol, {})

        for index, idx_stats in sbl_stats.items():
            rho_idx_stats = rho_sbl.get(index, 0)

            D[symbol].setdefault(index, 0)

            diff = abs(idx_stats - rho_idx_stats)

            D[symbol][index] += (rho_idx_stats - idx_stats) * eps
            diff_sum += diff

    for symbol, sbl_stats in RHO.items():
        if not HO.get(symbol):
            D.setdefault(symbol, {})

            HO.setdefault(symbol, {})

        for index, idx_stats in sbl_stats.items():
            if not HO[symbol].get(index):
                D[symbol].setdefault(index, idx_stats * eps)

                diff_sum += idx_stats
    
    return D, diff_sum



In [82]:
def get_history_stats(HistoryW, IDX, HO, inp, input_idx, h_idx):
    W = HistoryW[h_idx]
    idx = IDX[input_idx]

    O = W.get(inp + str(idx))

    for symbol, sbl_stats in O.items():
        HO.setdefault(symbol, {})

        for index, idx_stats in sbl_stats.items():
            HO[symbol].setdefault(index, 0)

            HO[symbol][index] += idx_stats

    return HO

In [83]:
def add_to_HW_update(HistoryW: list[dict[str, dict[str, dict[int, int]]]],
                     U: list[dict[str, dict[str, dict[int, int]]]],
                     IDX: list,
                     D: dict[str, dict[int, int]],
                     input: str,
                     inp_idx: int,
                     h_idx: int,
                     max_history: int, ):
    for h_idx in range(max_history):
        input_idx = inp_idx - h_idx - 1
        if input_idx < 0:
            break

        W = HistoryW[h_idx]
        idx = IDX[input_idx]

        key_I = input[input_idx] + str(idx)

        O = W.get(key_I)

        U[h_idx].setdefault(key_I, {})

        for symbol, sbl_stats in O.items():
            U[h_idx][key_I].setdefault(symbol, {})

            for index, idx_stats in sbl_stats.items():
                U[h_idx][key_I][symbol].setdefault(index, 0)

                U[h_idx][key_I][symbol][index] += idx_stats * D[symbol][index]

    return U


In [84]:
def add_to_RHW_update(RevHistoryW: list[dict[str, dict[str, dict[int, int]]]],
                      RU: list[dict[str, dict[str, dict[int, int]]]],
                      IDX: list,
                      D: dict[str, dict[int, int]],
                      input: str,
                      inp_idx: int,
                      h_idx: int,
                      max_history: int,
                      max_len: int,
                      ):
    for h_idx in range(max_history):
        input_idx = inp_idx + h_idx + 1
        if input_idx > max_len:
            break

        W = RevHistoryW[h_idx]
        idx = IDX[input_idx]

        key_I = input[input_idx] + str(idx)
        # print(f'{key_I=}')

        O = W.get(key_I)

        RU[h_idx].setdefault(key_I, {})

        for symbol, sbl_stats in O.items():
            RU[h_idx][key_I].setdefault(symbol, {})

            for index, idx_stats in sbl_stats.items():
                RU[h_idx][key_I][symbol].setdefault(index, 0)

                RU[h_idx][key_I][symbol][index] += idx_stats * -D[symbol][index]

    return RU


In [85]:
def update_HW(HW: list[dict[str, dict[str, dict[int, int]]]],
              U: list[dict[str, dict[str, dict[int, int]]]], 
              max_history: int, ):
    for h_idx in range(max_history):
        W = HW[h_idx]

        for I, O in W.items():
            if not U[h_idx].get(I):
                # print(f'Warn: U not contains: {I=}')
                break

            for symbol, sbl_stats in O.items():
                for index, idx_stats in sbl_stats.items():
                    update = U[h_idx][I][symbol][index]
                    
                    HW[h_idx][I][symbol][index] += update
    
    return HW

In [86]:
def reverse_feedback(input: str,
                     IDX: list,
                     HistoryW: list[dict[str, dict[str, dict[int, int]]]],
                     RevHistoryW: list[dict[str, dict[str, dict[int, int]]]],
                     max_history=4,
                     eps = 0.1, ):
    U: list[dict[str, dict[str, dict[int, int]]]] = []
    for hist_idx in range(max_history):
        U.append({})
    
    RU: list[dict[str, dict[str, dict[int, int]]]] = []
    for hist_idx in range(max_history):
        RU.append({})

    DT = []

    for inp_idx, inp in enumerate(input):
        if inp_idx == 0 or inp_idx > (len(input) - 1):
            continue
        # print(f'{inp_idx=}')

        HO: dict[str, dict[int, int]] = {}

        for h_idx in range(max_history):
            input_idx = inp_idx - h_idx - 1
            if input_idx < 0:
                break

            HO = get_history_stats(HistoryW, IDX, HO, input[input_idx], input_idx, h_idx)

        RHO: dict[str, dict[int, int]] = {}

        for h_idx in range(max_history):
            input_idx = inp_idx + h_idx + 1
            if input_idx > (len(input) - 1):
                break

            # print(f'{input_idx=} {h_idx=}')

            RHO = get_history_stats(RevHistoryW, IDX, RHO, input[input_idx], input_idx, h_idx)

        D, diff = HO_mx_diff(HO=HO, RHO=RHO, eps=eps)
        DT.append(diff)

        U = add_to_HW_update(HistoryW, U, IDX, D, input, inp_idx, h_idx, max_history, )
        RU = add_to_RHW_update(RevHistoryW, RU, IDX, D, input, inp_idx, h_idx, max_history, (len(input) - 1))
    
    HistoryW = update_HW(HW=HistoryW, U=U, max_history=max_history)

    RevHistoryW = update_HW(HW=RevHistoryW, U=RU, max_history=max_history)

    return HistoryW, RevHistoryW, DT

In [87]:
def plot_T(T: list):
    T_arr = np.array(T, dtype=float)

    fig, axs = plt.subplots(1, 1, figsize=(50, 5), tight_layout=True)
    axs.plot(np.arange(T_arr.size), T_arr)


    plt.show()

In [88]:
def predict(input_str: str, 
            HistoryW: list[dict[str, dict[str, dict[int, int]]]], 
            max_history = 4,
            max_index = 4,):
    err = 0
    TF = []
    AL = []

    history_indices = []
    history_indices.append([x for x in range(max_index)])

    for input_idx, input in enumerate(input_str[:-1]):
        HO: dict[str, dict[int, int]] = {}
        HO_cnt: dict[str, dict[int, int]] = {}

        itr_max_history = 0

        for h_idx in range(max_history):
            # print(f'{h_idx=}')

            inp_idx = input_idx - h_idx
            if inp_idx < 0:
                break

            itr_max_history += 1
            # print(f'{h_idx=} {inp_idx=}')
            W = HistoryW[h_idx]
            inp = input_str[inp_idx]
            indices = history_indices[h_idx]

            key_I = inp + str(indices[0])
            O = W.get(key_I, {})
            # print(f'{O=}')
            # print()

            for index in indices[1:]:
                key_I = inp + str(index)

                index_O = W.get(key_I, {})
                # print(f'{index_O=}')
                # print()

                for key, val in index_O.items():
                    O.setdefault(key, {})
                    O[key].update(val)
            
            # print(f'{O=}')
            # print()
            
            for key_O, val_O in O.items():
                HO.setdefault(key_O, {})
                HO_cnt.setdefault(key_O, {})
                
                for key_index, val_index in val_O.items():
                    HO[key_O].setdefault(key_index, 0)
                    HO_cnt[key_O].setdefault(key_index, 0)

                    # discont = (max_history - h_idx)
                    # discont = pow(0.1, h_idx)
                    discont = pow(2, (max_history - h_idx))

                    HO[key_O][key_index] += val_index * discont
                    HO_cnt[key_O][key_index] += 1

        # print(f'{HO=}')
        # print(f'{HO_cnt=}')
        # print()

        length = 0
        for v in HO.values():
            length += len(v)

        max_arr = []
        for v in HO.values():
            col_max = max(v, key=v.get)
            max_arr.append(col_max)
        
        max_arr = np.array(max_arr)
        # tf = np.mean(np.abs(max_arr - max_arr[np.newaxis].T))

        tf = length
        # tf = len(HO)

        TF.append(tf)

        # max by cnt
        # max_symbol = 'a'
        # max_symbol_index = 0
        # max_val = 0
        # for key, val in HO_cnt.items():
        #     max_idx = max(val, key=val.get)
        #     # HO_cnt[key] = {max_idx: val[max_idx]}

        #     if val[max_idx] > max_val: # 
        #         max_val = val[max_idx]
        #         max_symbol_index = max_idx
        #         max_symbol = key
        # print(f'{max_symbol=}')
        # max by val
        max_symbol = 'a'
        max_symbol_index = 0
        max_val = 0
        # win_cnt = 0
        for key, val in HO.items():
            for index, index_stat in val.items():
                index_cnt = HO_cnt[key][index]
                stat = pow(0.1, (itr_max_history - index_cnt)) * index_stat
                # stat = -0.1 * (itr_max_history - index_cnt) + index_stat

                HO[key][index] = stat

            max_idx = max(val, key=val.get)
            HO[key] = {max_idx: val[max_idx]}

            # max_cnt = HO_cnt[key][max_idx]

            if val[max_idx] > max_val: # and max_cnt >= max_his:
                # print(f'{key=} {val[max_idx]=} {max_cnt=}')
                max_val = val[max_idx]
                max_symbol_index = max_idx
                max_symbol = key
                # win_cnt = max_cnt

        # print(f'{HO=}')
        next_inp = input_str[input_idx + 1]
        # print(f'{max_symbol=}', f'{next_inp=} {max_val=} {itr_max_history=} {HO=}', max_symbol_index, max_val)
        # print('')

        # anomaly
        if max_symbol == next_inp:
            AL.append(0)
        else:
            ni_v = HO.get(next_inp, 0)
            if ni_v == 0:
                AL.append(100)
            else:
                max_ni = max(ni_v, key=ni_v.get)
                AL.append(abs(ni_v[max_ni] - max_val))

        new_indices = [max_symbol_index]

        # err
        if max_symbol != next_inp:
            err += 1

            index_stats = HO.get(next_inp, 0)
            
            if index_stats == 0:
                new_indices = [x for x in range(max_index)]
                print(f'{new_indices=}')
            else:
                max_symbol_index = list(index_stats.keys())[0]

                new_indices = [max_symbol_index]
        
        # recurtion
        history_indices.insert(0, new_indices)
        if len(history_indices) > max_history:
            history_indices = history_indices[:-1]

    TF.append(0)
    AL.append(0)

    return err, TF, AL

In [89]:
# print(f'______{IDX=}')
# print(f'{input_str=}')

err_L1, TF_L1, AL_L1 = predict(input_str, HistoryW, max_index=max_index_L1, max_history=max_history_L1)
print(err_L1, err_L1 / (len(input_str) / 100))

eps0 = 0.05
iters = 10

for iter_idx in range(iters):
    # eps = (iters - iter_idx) * eps0
    eps = eps0
    print(f'{iter_idx=} {eps=}')

    HistoryW, RevHistoryW, DT = reverse_feedback(input=input_str,
                                                 IDX=IDX,
                                                 HistoryW=HistoryW,
                                                 RevHistoryW=RevHistoryW,
                                                 max_history=max_history_L1,
                                                 eps=eps,)

    mean_DT = np.sum(np.array(DT))
    print(f'{mean_DT=}')
    # plot_T(T=DT)

    HistoryW = to_probability(HistoryW, max_history=max_history_L1)
    RevHistoryW = to_probability(RevHistoryW, max_history=max_history_L1)

    err_L1, TF_L1, AL_L1 = predict(input_str, HistoryW, max_index=max_index_L1, max_history=max_history_L1)
    print(err_L1, err_L1 / (len(input_str) / 100))


573 11.46
iter_idx=0 eps=0.05
mean_DT=31647.226512245208
534 10.68
iter_idx=1 eps=0.05
mean_DT=31456.81810334266
526 10.52
iter_idx=2 eps=0.05
mean_DT=31327.221204502108
536 10.72
iter_idx=3 eps=0.05
mean_DT=31229.56197704186
526 10.52
iter_idx=4 eps=0.05
mean_DT=31152.59829761262
529 10.58
iter_idx=5 eps=0.05
mean_DT=31089.732383360497
527 10.54
iter_idx=6 eps=0.05
mean_DT=31037.359764950892
529 10.58
iter_idx=7 eps=0.05
mean_DT=30992.793071391592
531 10.62
iter_idx=8 eps=0.05
mean_DT=30954.570495305587
533 10.66
iter_idx=9 eps=0.05
mean_DT=30921.449474048837
535 10.7


In [90]:
def get_word_stats(input_str: str | list, TF: list, threshold=0.0):
    buffer: list = [input_str[0]]
    WordStats: dict[str, int] = {}
    WordIndex: dict[str, int] = {}
    next_wrd_idx = 0

    NextLayerOut = []
    DownstreamIndex = {}

    prev_tf = TF[0]

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 and inp_idx >= len(input_str) - 1:
            continue

        # if TF[inp_idx] - prev_tf > 0.0: # and abs(TF[inp_idx] - prev_tf) > 0.2:
        # and TF[inp_idx] > threshold:
        if TF[inp_idx] > prev_tf and TF[inp_idx] > TF[inp_idx + 1]:
            word = ''.join(buffer)

            WordStats.setdefault(word, 0)
            WordStats[word] += 1

            if not WordIndex.get(word):
                WordIndex.setdefault(word, next_wrd_idx)
                next_wrd_idx += 1

            wrd_idx = WordIndex.get(word)
            # NextLayerOut.append(str(wrd_idx))
            NextLayerOut.append(word)

            DownstreamIndex.setdefault(inp_idx, 1)

            buffer = [inp]
            prev_tf = TF[inp_idx]

            continue

        TF[inp_idx] = 0

        buffer.append(inp)
        prev_tf = TF[inp_idx]

    # sort
    WordStats = dict(sorted(WordStats.items(), key=lambda item: item[1], reverse=True))

    NextLayerOut.append(str(next_wrd_idx))
    return WordStats, WordIndex, NextLayerOut, DownstreamIndex


TF_L1 = np.array(TF_L1, dtype='float64')
AL_L1 = np.array(AL_L1, dtype='float64')
TF_L1 /= np.max(TF_L1)
AL_L1 /= np.max(AL_L1)
TF_L1 += AL_L1

TF_max_coef = 0.4

# TF_L1[np.where(TF_L1 < np.mean(TF_L1))] = 0
# TF_1 = TF_L1 - np.pad(TF_L1, (1, 0))[:-1]
# TF_2 = TF_L1 - np.pad(TF_L1, (0, 1))[1:]
# TF_L1 = TF_1 + TF_2

# TF_L1[np.where(TF_L1 < 0)] = 0
# TF_L1[np.where(TF_L1 < TF_max_coef)] = 0
TF_L1 = TF_L1.tolist()

WordStats_L1, WordIndex_L1, NextLayerOut_L1, DownstreamIndex_L1 = get_word_stats(input_str=input_str,
                                                                                 TF=TF_L1, 
                                                                                 threshold=0.0, )
print(len(WordStats_L1))
mean_WS = np.mean(np.array(list(WordStats_L1.values())))
print(f'{mean_WS=}')
WS_len = 0
for key, val in WordStats_L1.items():
    # if len(key.strip()) > 1 and val >= mean_WS:
    if val >= mean_WS:
        WS_len += 1
    print(key, val)
print(f'{WS_len=}')

print(f'{NextLayerOut_L1=}')
print(f'{len(NextLayerOut_L1)=}')

print(f'{WordIndex_L1=}')

print(f'{DownstreamIndex_L1=}')


671
mean_WS=2.5752608047690013
 th 45
 he 32
 a 22
nd 21
 an 21
 t 19
id 19
ng 17
 sa 17
 b 16
 of 16
st 15
ing 15
 a  15
er 14
 so 14
 it 13
 to 13
 wh 12
 s 12
 be 11
ly 11
 wi 10
he 10
 bu 10
 yo 10
 is 9
 no 9
oph 9
 w 9
me 9
at 9
 i  9
ll 9
 go 9
 in 8
nnie 8
 on 8
 the 8
 h 8
 ro 8
 c 8
 l 8
 f 8
ri 8
e p 7
ere 7
 do 7
ad 7
 ch 7
bin 7
 as 7
ow 7
 y 7
 d 7
ou 7
 wa 7
e  7
th 7
li 7
an 6
ar 6
 co 6
bu 6
 o 6
d th 6
 i 6
ame 6
 al 6
 fr 6
ise 6
ed  6
oon 6
rs 5
om 5
en  5
re 5
 at 5
 re 5
ooh 5
sa 5
be 5
ie 5
 hi 5
 ti 5
er  5
zzing 5
se 5
ki 5
 ho 5
 we 4
nt 4
mp 4
is 4
rist 4
ay 4
 fe 4
hat 4
 and 4
od 4
ho 4
oh 4
 me 4
ca 4
 p 4
 g 4
out 4
ry 4
ve 4
 ha 4
en 4
ace 4
im 4
et 4
zz 4
 if 4
ney 4
on 4
 cl 4
imb 4
id  4
bo 4
ha 3
ro 3
 st 3
mi 3
nly 3
ut 3
 r 3
if 3
 fo 3
't 3
ou  3
wi 3
 ju 3
u  3
n't 3
e's 3
s  3
 q 3
t  3
ory 3
rt 3
 up 3
 by 3
lf 3
e n 3
tt 3
su 3
aid 3
e t 3
pa 3
d be 3
hi 3
nk 3
 m 3
of 3
ot 3
as 3
d a  3
ju 3
anch 3
ur 3
t h 3
 mo 3
ba 3
 ba 3
ooh  2
ories 2
w